In [5]:
#https://stackoverflow.com/questions/5041008/how-to-find-elements-by-class
#https://stackoverflow.com/questions/41447527/beautifulsoup-not-getting-all-html
#https://stackoverflow.com/questions/18791882/how-to-make-program-go-back-to-the-top-of-the-code-instead-of-closing
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup, NavigableString, Tag
import re
import time
import random
pd.set_option('display.max_colwidth', None)

In [9]:
book = 2
url = f'https://www.goodreads.com/book/show/{book}'
res = requests.get(url)
print(res.status_code)

200


In [10]:
soup = BeautifulSoup(res.content, 'html5lib')

In [110]:
#body = [item for item in soup.find_all('div', {'id':'metacol'})]
#body = soup.find(name='div', attrs={'id':'metacol'})
#body = [item.text for item in soup.find_all('h1')]

In [11]:
body = soup.find('h1')

In [12]:
body.text #title

'\n      Harry Potter and the Order of the Phoenix\n'

In [13]:
body = soup.find('span', {'itemprop':'ratingValue'})
body.text #number of stars (rating)

'\n  4.50\n'

In [15]:
body = soup.find('span', {'id':'freeText10736653061588882282', 'style':'display:none'})
body.text #synopsis

'There is a door at the end of a silent corridor. And itâ€™s haunting Harry Pottterâ€™s dreams. Why else would he be waking in the middle of the night, screaming in terror?Harry has a lot on his mind for this, his fifth year at Hogwarts: a Defense Against the Dark Arts teacher with a personality like poisoned honey; a big surprise on the Gryffindor Quidditch team; and the looming terror of the Ordinary Wizarding Level exams. But all these things pale next to the growing threat of He-Who-Must-Not-Be-Named - a threat that neither the magical government nor the authorities at Hogwarts can stop.As the grasp of darkness tightens, Harry must discover the true depth and strength of his friends, the importance of boundless loyalty, and the shocking price of unbearable sacrifice.His fate depends on them all.'

In [116]:
body = soup.find('span', {'itemprop':'name'})
body.text #author

'J.K. Rowling'

In [146]:
body = [item for item in soup.find_all('a', {'class':'actionLinkLite bookPageGenreLink'})]  
[i.text for i in body[:3]]  #top 3 genres

['Fantasy', 'Young Adult', 'Fiction']

In [16]:
def book_scrape(book_num):
    url = f'https://www.goodreads.com/book/show/{book_num}'
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html5lib')
    status = res.status_code
    while status == 403: #keep trying until either 200 or 404
        time.sleep(5) #wait 5 seconds
        url = f'https://www.goodreads.com/book/show/{book_num}' #try same number again
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'html5lib')
        status = res.status_code #check status code
    if status == 200:
        body = soup.find('h1')
        title = body.text #title
        title = title.replace('\n', '') #remove new line characters
        body = soup.find('span', {'itemprop':'ratingValue'})
        rating = body.text #number of stars (rating)
        rating = rating.replace('\n', '') #remove new line characters
        body = soup.find('div', {'id':'descriptionContainer'})
        body = soup.find('span', {'id':'freeText10736653061588882282', 'style':'display:none'})
        synopsis = body.text #synopsis
        body = soup.find('span', {'itemprop':'name'})
        author = body.text #author
        body = [item for item in soup.find_all('a', {'class':'actionLinkLite bookPageGenreLink'})]  
        genres = [i.text for i in body[:3]]  #top 3 genres
        genres = ', '.join(genres) #make string from list
    elif status == 404: #create empty row if page does not exist, will show book # and status code
        title, author, synopsis, genres, rating = np.nan, np.nan, np.nan, np.nan, np.nan 
    
    new_df = pd.DataFrame([book_num, status, title, author, synopsis, genres, rating],#aggregate scraped info into a dataframe
                      index=['book_number', 'status', 'title', 'author', 'synopsis', 'genres', 'rating']) #name df columns
    return new_df.T #need to transpose

In [17]:
book_scrape(2) #test

,book_number,status,title,author,synopsis,genres,rating
0,2,200,Harry Potter and the Order of the Phoenix,J.K. Rowling,"There is a door at the end of a silent corridor. And itâ€™s haunting Harry Pottterâ€™s dreams. Why else would he be waking in the middle of the night, screaming in terror?Harry has a lot on his mind for this, his fifth year at Hogwarts: a Defense Against the Dark Arts teacher with a personality like poisoned honey; a big surprise on the Gryffindor Quidditch team; and the looming terror of the Ordinary Wizarding Level exams. But all these things pale next to the growing threat of He-Who-Must-Not-Be-Named - a threat that neither the magical government nor the authorities at Hogwarts can stop.As the grasp of darkness tightens, Harry must discover the true depth and strength of his friends, the importance of boundless loyalty, and the shocking price of unbearable sacrifice.His fate depends on them all.","Fantasy, Young Adult, Fiction",4.50


In [179]:
book_scrape(20) #testing problem book number

,book_number,status,title,author,synopsis,genres,rating
0,20.0,404.0,NaN,NaN,NaN,NaN,NaN


``` Python
test_books = list(range(1,10_001)) #list of numbers 1-10,000
main_df = pd.DataFrame() #create empty data frame
for n in test_books:
    main_df = pd.concat([main_df, book_scrape(n)])
    time.sleep(random.randint(2,4)) #random delay after each loop
    if len(main_df) % 10 == 0: #save every 10 rows
        main_df.to_csv('./data/10k_test_scrape.csv', index=False)
        if len(main_df) % 1_000 == 0:
            print(f'Current rows: {len(main_df)}; Scraping {len(main_df)/100}% complete') #print status update every 1k rows
main_df.to_csv('./data/10k_test_scrape.csv', index=False) #save final df
main_df #preview final df    
```

In [172]:
book_scrape(9997) #testing values that came out null in 10k test

200


,book_number,title,author,synopsis,genres,rating
0,9997,The Ruined Map,Kōbō Abe,"Of all the great Japanese novelists, Kobe Abe was indubitably the most versatile. With The Ruined Map, he crafted a mesmerizing literary crime novel that combines the narrative suspense of Chandler with the psychological depth of Dostoevsky.Mr. Nemuro, a respected salesman, disappeared over half a year ago, but only now does his alluring yet alcoholic wife hire a private e Of all the great Japanese novelists, Kobe Abe was indubitably the most versatile. With The Ruined Map, he crafted a mesmerizing literary crime novel that combines the narrative suspense of Chandler with the psychological depth of Dostoevsky.Mr. Nemuro, a respected salesman, disappeared over half a year ago, but only now does his alluring yet alcoholic wife hire a private eye. The nameless detective has but two clues: a photo and a matchbook. With these he embarks upon an ever more puzzling pursuit that leads him into the depths of Tokyo’s dangerous underworld, where he begins to lose the boundaries of his own identity. Surreal, fast-paced, and hauntingly dreamlike, Abe’s masterly novel delves into the unknowable mysteries of the human mind.Translated from the Japanese by E. Dale Saunders.","Fiction, Cultural, Japan",3.53


In [167]:
book_scrape(25) #testing values that came out null in 10k test

,book_number,title,author,synopsis,genres,rating
0,25,I'm a Stranger Here Myself: Notes on Returning to America After Twenty Years Away,Bill Bryson,"After living in Britain for two decades, Bill Bryson recently moved back to the United States with his English wife and four children (he had read somewhere that nearly 3 million Americans believed they had been abducted by aliens--as he later put it, ""it was clear my people needed me""). They were greeted by a new and improved America that boasts microwave pancakes, twenty After living in Britain for two decades, Bill Bryson recently moved back to the United States with his English wife and four children (he had read somewhere that nearly 3 million Americans believed they had been abducted by aliens--as he later put it, ""it was clear my people needed me""). They were greeted by a new and improved America that boasts microwave pancakes, twenty-four-hour dental-floss hotlines, and the staunch conviction that ice is not a luxury item. Delivering the brilliant comic musings that are a Bryson hallmark, I'm a Stranger Here Myself recounts his sometimes disconcerting reunion with the land of his birth. The result is a book filled with hysterical scenes of one man's attempt to reacquaint himself with his own country, but it is also an extended if at times bemused love letter to the homeland he has returned to after twenty years away.","Nonfiction, Travel, Humor",3.90


In [173]:
#print(main_df.isnull().sum().to_markdown())

|             |    0 |
|:------------|-----:|
| book_number |    0 |
| title       | 9471 |
| author      | 9471 |
| synopsis    | 9471 |
| genres      | 9471 |
| rating      | 9471 |


#### Results from first attempt at 10k posts  

|  column     | is_null() |
|:------------|-----:|
| book_number |    0 |
| title       | 9471 |
| author      | 9471 |
| synopsis    | 9471 |
| genres      | 9471 |
| rating      | 9471 |

``` Python
#second test - 100 books and recording status number for each pull
test_books = list(range(1,101)) #list of numbers 1-10,000
main_df = pd.DataFrame() #create empty data frame
for n in test_books:
    main_df = pd.concat([main_df, book_scrape(n)])
    time.sleep(random.randint(2,4)) #random delay after each loop
    if len(main_df) % 10 == 0: #save every 10 rows
        main_df.to_csv('./data/100_test_scrape.csv', index=False)
        if len(main_df) % 1_000 == 0:
            print(f'Current rows: {len(main_df)}; Scraping {len(main_df)/100}% complete') #print status update every 1k rows
main_df.to_csv('./data/100_test_scrape.csv', index=False) #save final df
main_df #preview final df  
```

In [181]:
#main_df['status'].value_counts()

200.0    68
403.0    24
404.0     8
Name: status, dtype: int64

|http status  |   # |
|----:|---------:|
| 200 |       68 |
| 403 |        24 |
| 404 |  8  |

In [182]:
#main_df.isnull().sum()

book_number     0
status          0
title          32
author         32
synopsis       32
genres         32
rating         32
dtype: int64

``` Python
#third test - 100 books, attempting while loop to keep trying 403's
test_books = list(range(1,101)) #list of book numbers (according to goodreads)
main_df = pd.DataFrame() #create empty data frame
for n in test_books:
    main_df = pd.concat([main_df, book_scrape(n)])
    time.sleep(random.randint(2,4)) #random delay after each loop
    if len(main_df) % 10 == 0: #save every 10 rows
        main_df.to_csv('./data/100_test2_scrape.csv', index=False)
        if len(main_df) % 1_000 == 0:
            print(f'Current rows: {len(main_df)}; Scraping {len(main_df)/100}% complete') #print status update every 1k rows
main_df.to_csv('./data/100_test2_scrape.csv', index=False) #save final df
main_df #preview final df 
```

|  column    |   is_null() |
|:------------|----:|
| book_number |   0 |
| status      |   0 |
| title       |   8 |
| author      |   8 |
| synopsis    |   8 |
| genres      |   8 |
| rating      |   8 |

|http status  |   # |
|----:|---------:|
| 200 |       92 |
| 404 |        8 |

In [24]:
main_df

,book_number,status,title,author,synopsis,genres,rating
0,1,200,Harry Potter and the Half-Blood Prince,J.K. Rowling,"The war against Voldemort is not going well; even Muggle governments are noticing. Ron scans the obituary pages of the Daily Prophet, looking for familiar names. Dumbledore is absent from Hogwarts for long stretches of time, and the Order of the Phoenix has already suffered losses.And yet . . .As in all wars, life goes on. The Weasley twins expand their business. Sixth-yea The war against Voldemort is not going well; even Muggle governments are noticing. Ron scans the obituary pages of the Daily Prophet, looking for familiar names. Dumbledore is absent from Hogwarts for long stretches of time, and the Order of the Phoenix has already suffered losses.And yet . . .As in all wars, life goes on. The Weasley twins expand their business. Sixth-year students learn to Apparate - and lose a few eyebrows in the process. Teenagers flirt and fight and fall in love. Classes are never straightforward, through Harry receives some extraordinary help from the mysterious Half-Blood Prince.So it's the home front that takes center stage in the multilayered sixth installment of the story of Harry Potter. Here at Hogwarts, Harry will search for the full and complete story of the boy who became Lord Voldemort - and thereby find what may be his only vulnerability.","Fantasy, Young Adult, Fiction",4.57
0,2,200,Harry Potter and the Order of the Phoenix,J.K. Rowling,"There is a door at the end of a silent corridor. And itâ€™s haunting Harry Pottterâ€™s dreams. Why else would he be waking in the middle of the night, screaming in terror?Harry has a lot on his mind for this, his fifth year at Hogwarts: a Defense Against the Dark Arts teacher with a personality like poisoned honey; a big surprise on the Gryffindor Quidditch team; and the loomi There is a door at the end of a silent corridor. And itâ€™s haunting Harry Pottterâ€™s dreams. Why else would he be waking in the middle of the night, screaming in terror?Harry has a lot on his mind for this, his fifth year at Hogwarts: a Defense Against the Dark Arts teacher with a personality like poisoned honey; a big surprise on the Gryffindor Quidditch team; and the looming terror of the Ordinary Wizarding Level exams. But all these things pale next to the growing threat of He-Who-Must-Not-Be-Named - a threat that neither the magical government nor the authorities at Hogwarts can stop.As the grasp of darkness tightens, Harry must discover the true depth and strength of his friends, the importance of boundless loyalty, and the shocking price of unbearable sacrifice.His fate depends on them all.","Fantasy, Young Adult, Fiction",4.50
0,3,200,Harry Potter and the Sorcerer's Stone,J.K. Rowling,"Harry Potter's life is miserable. His parents are dead and he's stuck with his heartless relatives, who force him to live in a tiny closet under the stairs. But his fortune changes when he receives a letter that tells him the truth about himself: he's a wizard. A mysterious visitor rescues him from his relatives and takes him to his new home, Hogwarts School of Witchcraft Harry Potter's life is miserable. His parents are dead and he's stuck with his heartless relatives, who force him to live in a tiny closet under the stairs. But his fortune changes when he receives a letter that tells him the truth about himself: he's a wizard. A mysterious visitor rescues him from his relatives and takes him to his new home, Hogwarts School of Witchcraft and Wizardry.After a lifetime of bottling up his magical powers, Harry finally feels like a normal kid. But even within the Wizarding community, he is special. He is the boy who lived: the only person to have ever survived a killing curse inflicted by the evil Lord Voldemort, who launched a brutal takeover of the Wizarding world, only to vanish after failing to kill Harry.Though Harry's first year at Hogwarts is the best of his life, not everything is perfect. There is a danger

In [26]:
main_df['genres'].value_counts()

                                                     8
Fantasy, Classics, Fiction                           6
Fantasy, Young Adult, Fiction                        6
Science Fiction, Fiction, Humor                      6
Historical, Historical Fiction, Fiction              5
Travel, Nonfiction, Humor                            4
Nonfiction, Writing, Essays                          4
Fantasy, Fiction, Young Adult                        3
Christian, Biography, Nonfiction                     2
Education, Teaching                                  2
Young Adult, Fiction, Adventure                      2
Science Fiction, Audiobook, Humor                    2
Business                                             1
Fantasy, Fiction, Classics                           1
Art, Crafts, Crafts                                  1
Nonfiction, History, Humanities                      1
Fiction, Contemporary                                1
Fantasy, Young Adult, Romance                        1
Nonfiction